In [45]:
import h5py, sys
import numpy as np
import pandas as pd
with h5py.File(r"Z:\OneDrive_1_11-9-2022\HydraulicsBuild_MB.p02.hdf",'r') as hf:
    dataset_names = list(hf.keys())
    # with open('filename.txt', 'w') as f:
    #     sys.stdout = f # Change the standard output to the file we created.
    #     hf.visit(print)


In [2]:
dataset_names

['Event Conditions', 'Geometry', 'Plan Data', 'Results', 'Sediment']

In [49]:
hf = h5py.File(r"Z:\OneDrive_1_11-9-2022\HydraulicsBuild_MB.p02.hdf",'r')
hf['/Geometry/Cross Sections/Polyline Parts']

<HDF5 dataset "Polyline Parts": shape (490, 2), type "<i4">

In [26]:
poly_parts = hf['/Geometry/Cross Sections/Polyline Parts'][:,1]

np.sum(poly_parts)

1237

In [ ]:
# '/Geometry/Cross Sections/Polyline Parts' gives the number of vertices of each XSection.
# Total parts across 490 XSections = 1237

# Each Polyline Part has a coordinate in '/Geometry/Cross Sections/Polyline Points'/

# Dataframe Dimensions (XSection Index, XSection Name, XSection Geometry)

# XSection Name = '/Results/Unsteady/Geometry Info/Cross Section Only'


In [ ]:
# Xsection 0 = Old River Floodplain 60845.5, has 3 polyline parts, [(1360127.107, 2889357.839), (1359439.607, 2888899.506), (1356393.834, 2888635.568)] 

In [27]:
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point

bounds = [(1360127.107, 2889357.839), (1359439.607, 2888899.506), (1356393.834, 2888635.568)]
line_geom = LineString(bounds)
polygon = gpd.GeoDataFrame(index=[0], geometry=[line_geom])
polygon.crs = 'PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["false_easting",0.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",-96.0],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["latitude_of_origin",23.0],UNIT["Foot_US",0.3048006096012192]]'


In [43]:
import folium
m = folium.Map([30.2, -90.3], zoom_start=8, tiles='cartodbdark_matter')
folium.GeoJson(data=polygon, tooltip='Old River Floodplain 60845.5', ).add_to(m)
# folium.LatLngPopup().add_to(m)

In [44]:
m.save('geometry.html')

In [73]:
# Dataframe Dimensions (XSection Index, XSection Name, XSection Geometry)
df['XS_Name'] = hf['/Results/Unsteady/Geometry Info/Cross Section Only'][:]
# df['Geometry'] = df_geometry

In [74]:
eventList = []
m = folium.Map([30.2, -90.3], zoom_start=8, tiles='cartodbdark_matter')
coord_index_start = 0
for i,v in enumerate(df['XS_Name']):
    # pick style based on even or odd index
    if (i % 2) == 0:
        style = {"color": "#228B22", "weight": 5, "opacity": 0.65}
    else:
        # style = {'fillColor': '#00FFFFFF', 'color': '#00FFFFFF'}
        style = {"color": "#ff7800", "weight": 5, "opacity": 0.65}
    parts = hf['/Geometry/Cross Sections/Polyline Parts'][i,1]
    # the index range of coordinates to pull based on the number of parts for each XS.
    coord_index_end = coord_index_start + parts
    coords_range = [coord_index_start, coord_index_end]
    bounds = [hf['/Geometry/Cross Sections/Polyline Points'][coords_range].tolist()][0]
    # update start of range for next XS in loop.
    coord_index_start =+ parts
    line_geom = LineString(bounds)
    line = gpd.GeoDataFrame(index=[0], geometry=[line_geom])
    line.crs = 'PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["false_easting",0.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",-96.0],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["latitude_of_origin",23.0],UNIT["Foot_US",0.3048006096012192]]'
    folium.GeoJson(data=line, tooltip=v, style_function=lambda x:style).add_to(m)

In [75]:
m.save('hdf_XS_map.html')